### classify stackoverflow posts by convoluting over its individual word embeddings

**TODO**
 - use weighted averages (tf-id, ppmi, etc)
 - use a different scoring function so that wrong guesses that are close to the correct values (think "python", "python-2") are penalized less than guesses that are wildly wrong.


In [1]:
import csv
import os
import re
import sys
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Input, Flatten, Activation
from keras.layers import Conv1D, MaxPooling1D, Embedding, GlobalAvgPool1D
from keras.models import Model, Sequential

from sklearn import preprocessing
from sklearn.feature_extraction.text import CountVectorizer

# my stuff in the helpers/ directory
from helpers import files_helper, texts_helper, metrics_helper, tags_helper

Using TensorFlow backend.


In [2]:
np.random.seed(seed=42)

In [3]:
texts, labels = files_helper.read_stackoverflow_sample_small()

In [4]:
preprocessed_texts = texts

In [5]:
MAX_NB_WORDS = 20000
MAX_SEQUENCE_LENGTH = 1000
VALIDATION_SPLIT = 0.2
EMBEDDING_DIM=100
LABELS_MIN_DOC_COUNT = int(10)

In [6]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS,
                     filters='\'!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(preprocessed_texts)
sequences = tokenizer.texts_to_sequences(preprocessed_texts)
word_index = tokenizer.word_index
inverse_word_index = texts_helper.build_inverse_word_index(word_index)

In [7]:
[inverse_word_index[idx] for idx in sequences[0]]

['apply',
 'onclick',
 'event',
 'to',
 'an',
 'option',
 'i',
 'am',
 'using',
 'zend',
 'form',
 'to',
 'create',
 'a',
 'form',
 'i',
 'am',
 'also',
 'using',
 'for',
 'javascript',
 'this',
 'gt',
 'radio',
 'alone',
 'array',
 'label',
 'gt',
 'are',
 'you',
 'going',
 'to',
 'be',
 'taking',
 'part',
 'with',
 'anyone',
 'else',
 'required',
 'gt',
 'true',
 'onclick',
 'gt',
 'gt',
 'array',
 'yes',
 'gt',
 'yes',
 'no',
 'gt',
 'no',
 'at',
 'the',
 'moment',
 'the',
 'onclick',
 'event',
 'works',
 'if',
 'any',
 'option',
 'is',
 'selected',
 'how',
 'do',
 'i',
 'get',
 'it',
 'to',
 'work',
 'for',
 'just',
 'yes',
 'being',
 'selected']

In [8]:
truncated_labels = tags_helper.truncate_labels(labels,LABELS_MIN_DOC_COUNT)

In [9]:
lb = preprocessing.MultiLabelBinarizer()
binary_labels = lb.fit_transform(truncated_labels)

In [10]:
# tag position => fraction of docs having that tag
tag_probabilities_index = tags_helper.get_probabilities_index(binary_labels)

In [11]:
# word => embedding
embeddings_index = files_helper.read_glove(d=EMBEDDING_DIM)

In [12]:
data = pad_sequences(sequences,maxlen=MAX_SEQUENCE_LENGTH)

indices = np.arange(data.shape[0])
np.random.shuffle(indices)

data = data[indices]
labels_1 = binary_labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

X_train = data[:-num_validation_samples]
Y_train = labels_1[:-num_validation_samples]
X_val = data[-num_validation_samples:]
Y_val = labels_1[-num_validation_samples:]

data.shape,labels_1.shape

((40000, 1000), (40000, 1469))

In [13]:
embedding_matrix = np.zeros((len(word_index)+1,EMBEDDING_DIM))

for word,i in word_index.items():
    
    if i >= MAX_NB_WORDS:
        continue
    
    embedding_vector = embeddings_index.get(word)
    
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [14]:
%%time

num_labels = labels_1.shape[1]

model = Sequential()

model.add(Embedding(len(word_index)+1,
                           EMBEDDING_DIM,
                           weights=[embedding_matrix],
                           input_length=MAX_SEQUENCE_LENGTH,
                           trainable = False))

model.add(Conv1D(128,5,activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(Conv1D(128,5,activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(GlobalMaxPool1D())
model.add(Dense(num_labels))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam')
# multi label needs a specific accuracy function


# loss doesn't get better after 5 epochs
model.fit(X_train, Y_train,
                    batch_size=32,
                    epochs=5,
                    verbose=1,
                    validation_data=(X_val,Y_val))

Train on 32000 samples, validate on 8000 samples
Epoch 1/5
32000/32000 [==============================] - 43s - loss: 0.0228 - val_loss: 0.0099
Epoch 2/5
32000/32000 [==============================] - 36s - loss: 0.0099 - val_loss: 0.0099
Epoch 3/5
32000/32000 [==============================] - 36s - loss: 0.0099 - val_loss: 0.0099
Epoch 4/5
32000/32000 [==============================] - 37s - loss: 0.0099 - val_loss: 0.0099
Epoch 5/5
32000/32000 [==============================] - 37s - loss: 0.0098 - val_loss: 0.0098
CPU times: user 2min 25s, sys: 24.2 s, total: 2min 49s
Wall time: 3min 21s


In [67]:
def get_predicted_indices_by_tag_doc_fraction(
    tag_probabilities_index,
    predicted_tag_probabilities,
    multiplier=1.0,
    relative_difference_threshold=None):
    """
    
    (if predicted_index is not in the index, assume the average over all tags
    """
       
    output = []

    # this can be put outside to save time
    sum_over_all_tags = sum([value for _,value in tag_probabilities_index.items()])

    # no need for float casting in python 3
    mean_over_all_tags = sum_over_all_tags / len(tag_probabilities_index)

    for i,prob in enumerate(predicted_tag_probabilities.ravel()):

        avg_prob = tag_probabilities_index.get(i)
        
        relative_difference = (prob-avg_prob)/avg_prob
        
        if avg_prob is None:
            if prob > mean_over_all_tags:
                output.append(1)
            else:
                output.append(0)
        elif relative_difference_threshold is None:
            if prob > avg_prob * multiplier:
                output.append(1)
            else:
                output.append(0)
        else:   
            if prob > avg_prob and relative_difference > relative_difference_threshold:
                output.append(1)
            else:
                output.append(0)

    return np.array(output).reshape(1,-1)

In [17]:
def get_top_k_predicted_indices(
    predicted_tag_probabilities,
    k=5):
    """
    
    (if predicted_index is not in the index, assume the average over all tags
    """
    
    
    tags_probs = [ (tag,prob) for tag,prob in enumerate(predicted_tag_probabilities.ravel())]
   
    sorted_tags_probs = sorted(tags_probs,key=lambda tpl:tpl[1],reverse=True)
       
    tags_ordered = [tag for tag,_ in sorted_tags_probs][:k]
    
    predicted_label_indices = np.zeros(len(tags_probs),dtype='int64')
       
    predicted_label_indices[tags_ordered] = 1   
       
    predicted_label_indices = predicted_label_indices.reshape(1,-1)
    
    return predicted_label_indices

In [21]:
def get_predicted_indices_by_threshold(predicted_tag_probabilities,
                                   threshold=0.08):
    
    predicted_label_indices= np.zeros(len(predicted_tag_probabilities),dtype='int64')
    
    predicted_label_indices[predicted_tag_probabilities > threshold] = 1

    return predicted_label_indices

In [70]:
# sample result for a couple of test cases
num_test_cases = 10
threshold=0.05
multiplier=20.0
relative_difference_threshold = None

for i in np.random.randint(low=0, high=len(Y_val), size=num_test_cases):

    actual_label_indices = Y_val[i].reshape(1,-1)
    predicted_tag_probabilities = model.predict(X_val[i].reshape(1,-1)).ravel()
                    
    predicted_label_indices = get_predicted_indices_by_tag_doc_fraction(
        tag_probabilities_index,
        predicted_tag_probabilities,
        multiplier=multiplier,
    relative_difference_threshold=relative_difference_threshold)
        
        
    actual_labels = lb.inverse_transform(actual_label_indices)
    
    predicted_labels = lb.inverse_transform(predicted_label_indices.reshape(1,-1))
    
    scores = predicted_tag_probabilities[predicted_label_indices.nonzero()[1]]
    
    print(actual_labels)
    print( list(zip(predicted_labels[0],scores)) )
    print('\n')


[('html', 'jquery')]
[('android-widget', 0.0022183086), ('c++-cli', 0.0022942973), ('emulator', 0.0023886724), ('html5-audio', 0.0016526591), ('lisp', 0.0032251477), ('port', 0.0026697055), ('protocols', 0.0020258422), ('refactoring', 0.0031415662), ('screen', 0.0027442609), ('tortoisesvn', 0.001613241)]


[('mysql',)]
[('android-widget', 0.0024906704), ('asmx', 0.0013895067), ('c++-cli', 0.0025593475), ('cocos2d-iphone', 0.0028913771), ('emulator', 0.002685356), ('html5-audio', 0.0017525568), ('lisp', 0.003288161), ('port', 0.0027290219), ('protocols', 0.0024493763), ('refactoring', 0.0031995317), ('screen', 0.0028625571), ('tortoisesvn', 0.0017926072), ('uwp', 0.003212092), ('vhdl', 0.0019120878), ('wifi', 0.002495531)]


[('asp.net', 'nhibernate')]
[('anchor', 0.0032107076), ('android-widget', 0.0022452103), ('asmx', 0.0014150558), ('c++-cli', 0.0028340102), ('emulator', 0.0026237445), ('internet-explorer-7', 0.0028248024), ('out-of-memory', 0.0017988536), ('protocols', 0.0021231712

In [71]:
X_test_list = []

for validation_features in X_val:
    X_test_list.append(validation_features.reshape(1,-1))

X_test = np.vstack(X_test_list)    

In [72]:
Y_pred_lst = [model.predict(x_test.reshape(1,-1)) for x_test in X_test]

Y_pred_calculated_lst = [get_predicted_indices_by_tag_doc_fraction(
    tag_probabilities_index,
    y_pred,
    multiplier=multiplier,
    relative_difference_threshold=relative_difference_threshold) for y_pred in np.vstack(Y_pred_lst)]

Y_pred = np.vstack(Y_pred_calculated_lst)

Y_val.shape,Y_pred.shape

micro_f1 = metrics_helper.calculate_multilabel_metrics(Y_val,Y_pred)
print('micro-averaged F1 score (validation set) is {0}'.format(micro_f1))

micro_prec = metrics_helper.calculate_multilabel_metrics(Y_val, Y_pred, metric='precision')
print('micro-averaged precision score (validation set) is {0}'.format(micro_prec))

micro_rec = metrics_helper.calculate_multilabel_metrics(Y_val, Y_pred, metric='recall')
print('micro-averaged recall score (validation set) is {0}'.format(micro_rec))

micro-averaged F1 score (validation set) is 0.0036984326984961517
micro-averaged precision score (validation set) is 0.0022317765597820735
micro-averaged recall score (validation set) is 0.010787942887361185
